In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
client = MongoClient()
db = client.eddies
collection = db.cchdo

In [3]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [4]:
rootdir = '/home/cz2397/data/cchdo-ctd/'

In [5]:
station = 0
overlap = 0
result = []

In [6]:
db.cchdo.create_index([('dat', pymongo.ASCENDING), ('loc', pymongo.ASCENDING)])

u'dat_1_loc_1'

In [19]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('ctd.nc'):
            url = os.path.join(subdir, file)
            DS = xr.open_dataset(url)
            DAT = DS.time.values[0]
            LAT = DS.latitude.values[0]
            LON = format(DS.longitude.values[0])
            dat_var = np.timedelta64(3, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            lat_var = 0.2
            lat_min = LAT - lat_var
            lat_max = LAT + lat_var
            lon_var = 0.4
            lon_min = LON - lon_var
            lon_max = LON + lon_var
            ft = {
                'dat': {'$gt': dat_min, '$lt': dat_max},
                'loc': {'$within': {'$box': [[lon_min, lat_min], [lon_max, lat_max]]}}
            }
            count = db.cchdo.find(ft).count()
            station = station + 1
            if count != 0:
                overlap = overlap + 1
                result.append(url)

100%|██████████| 115/115 [00:03<00:00, 33.30it/s]


In [20]:
station

86941

In [21]:
overlap

1351

In [22]:
result

['/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00016_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00015_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00032_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00109_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00097_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00093_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00058_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/707_ar13_a_nc_ctd/ar13_a_00052_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/745_35MF20091219_nc_ctd/35MF20091219_00006_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/610_ar19_c_nc_ctd/ar19_c_00043_00001_ct1.nc',
 '/home/cz2397/data/cchdo-ctd/505_pr03_c_nc_ctd/pr03_c_ry8318_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/483_p14_49NZ20071122_nc_ctd/p14C_49NZ20071122_00023_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/483_p14_49NZ20071122_nc_